<img style="float: right; padding: 10px;" width="200" src="images/logo-onem2m-the-iot-standard.jpg">

# oneM2M - Subscriptions and Notifications

This notebook demonstrates how to subscribe to notifications from resources.

<p>
<div  style="background-color:transparent;border:2px #005480 solid;padding: 10px;width: calc(100% - 350px);border-radius:10px;">
    <div style="color:#005480;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b></div>
    <div style="color:#545054;">
Please execute the notebook "oneM2M - Subscriptions and Notifications - Notification Server" from the <a href="start-notificationServer.ipynb">notification server notebook</a> first.<br>
This starts a notification server that will receive the notifications that result from the following requests.
<br>
If you later see an error like "internal server errror" and a <i>Result Status Code</i> of <i>5204</i> you most likely forget to start the notification server. Please open the <a href="start-notificationServer.ipynb">notification server notebook</a> and run or restart the server.    </div>
</div>
</p>

- Create a &lt;subscription> resource
- Create &lt;contentInstance> resource and receive a notification


## Intitialization
The section does import necessary modules and configurations, and prepares the CSE for this notebook.


In [ ]:
%run src/init.py subscriptions

## Subscribe to a &lt;container> Resource

<img align="right" src="images/06/create_subscription.png">



The following requests creates a &lt;subscription> resource for the *Container* resource.

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 600px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
&lt;subscription> resource are used to monitor resource creations, updates and more. They also carry the configuration when and how the notifications are sent to a notification target.
<br /><br />
Please note, that instead of specifying a direct URL in the &lt;subscription> we usually set an &lt;AE> resource as a target for a notification and then specify the URL in the &lt;AE>'s <i>point-of-access</i> (poa) attribute. This has the advantage that an &lt;AE> is in control of its notification endpoint instead of each &lt;subscription> resource.
    </div>
</div>
</p> 



In [ ]:
CREATE (                                           # CREATE request
    
    # Create subscription resource under the container
    to                      = cseBaseName + '/StreetLight-AE-2/Light-Container-2',  
    
    # Request Parameters
    originator              = 'CstreetLight-AE-1', # Set the originator
    requestIdentifier       = '123',               # Unique request identifier
    releaseVersionIndicator = '3',                 # Release version indicator
    resourceType            = Type.Subscription,   # Type of the resource: Subscription
    
    # Request Body
    primitiveContent = 
        {
            'm2m:Subscription': {
                'resourceName'  : 'Subscription',
                'notificationURI'  : [ notificationURL ],
                'notificationContentType' : 1,
                'eventNotificationCriteria' : {
                    'notificationEventType': [ 1, 2, 3, 4 ]
                }
            }
        }
)

**Please check the [Notification Server](start-notificationServer.ipynb) notebook** (the one with the running Notification Server) to see the verification request.  
When this requests returns with a *201 (Created)* status code / *2001 oneM2M Response Status Code* then the creation of the &lt;subscription> resource as well as the verification request were successfull.

<p>
<div  style="background-color:transparent;border:2px #005480 solid;padding: 10px;width: calc(100% - 600px);border-radius:10px;">
    <div style="color:#005480;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; Note</b></div>
    <div style="color:#545054;">
    If you see an error like "Verification request failed" or "internal server errror" and a <i>Response Status Code</i> of 5204 you most likely forget to start the notification server. Please open the <a href="start-notificationServer.ipynb">notification server</a> notebook and run the server.<br>
    <b>Then you need to execute the above cell again!</b>    
    </div>
</div>
</p>

<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 600px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
        A <i>verification request</i> is used by the CSE to verify that the target is reachable and willing to receive notifications. Any failure or rejection by the target server leads to a rejection of the CREATE request of the &lt;subscription>.
    </div>
</div>
</p>


### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute                 | Short Name | Description                                                                                                                                     |
|:--------------------------|:-----------|:------------------------------------------------------------------------------------------------------------------------------------------------|
| notificationURI           | nu         | The Notification URI specifies where to send verification and notification requests. This could be an entity, like an &lt;AE>, or a direct URL. |
| eventNotificationCriteria | enc        | The configuration for the &lt;subscription> that specifies when to send a event.                                                                |
| notificationContentType   | nct        | The Notification Content Type attribute specifies the kind of content that should be sent in a notification initiated by this subscription.     |
| notificationEventType     | net        | A list of Notification Event Types indicates the reason when a notification is sent.                                                            |
| notificationStatsEnable   | nse        | Record statistics per target how many notifications have been sent.                                                                             |
| notificationStatsInfo     | nsi        | A list of notification statistics per target.                                                                                                   |
| notification              | sgn        | A notification request structure.                                                                                                               |
| verificationRequest       | vrq        | Indicator that a notification is a so-called verification request.                                                                              |
| subscriptionReference     | sur        | The resource ID of the subscription for the notification request.                                                                               |
| creator                   | cr         | The originator of a request that caused the notification.                                                                                       |


#### Values for Notification Content Type

| Value | Interpretation                           |
|:------|:-----------------------------------------|
| 1     | All. The whole resource                  |
| 2     | Only modified attributes                 |
| 3     | The resource's *resource ID*             |
| 4     | The trigger payload in case of Triggers  |


#### Notification Event Types


| Value | Interpretation                                        |
|:------|:------------------------------------------------------|
| 1     | Update of Resource                                    |
| 2     | Delete of Resource                                    |
| 3     | Create of Direct Child Resource                       |
| 4     | Delete of Direct Child Resource                       |
| 5     | Retrieve of Container Resource With No Child Resource |
| 6     | Trigger Received For AE_Resource                      |
| 7     | Blocking Update                                       |



<img align="right" src="images/06/create_contentInstance.png">


## Add a &lt;contentInstance> Resource to Light-Container and Send a Notification

We now have subscribed to a status change, i.e. whenever a child resource is added to the &lt;container>.
For example, when Streetlight-AE-2 receives a new command under this container, then a notification is raised by the &lt;subscription> resource.

This is shown with the following request. To simplify he example the AE "StreetLight-AE-1" is the reques sender.

In [ ]:
CREATE (                                             # CREATE request
    
    # Add the <container> under the <AE>
    to                      = cseBaseName + '/StreetLight-AE-2/Light-Container-2',
    
    # Request Parameters
    originator              = 'CstreetLight-AE-1',   # Set the originator
    requestIdentifier       = '123',                 # Unique request identifier
    releaseVersionIndicator = '3',                   # Release version indicator
    resourceType            = Type.ContentInstance,  # Type of the resource (ContentInstance)
    
    # Request Body
    primitiveContent = 
        {
            'm2m:ContentInstance' : {
                'contentInfo' : 'text/plain:0',
                'content' : 'lights on'
            }
        }
)

**Again, please check the [Notification Server](start-notificationServer.ipynb) notebook to see the notification.** 


<p>
<div style="background-color:transparent;border:2px #b42025 solid;padding: 10px;width: calc(100% - 600px);border-radius:10px;">
    <div style="color:#b42025;"><b><i class="fa fa-info-circle" aria-hidden="true"></i>&nbsp; oneM2M</b></div>
    <div style="color:#545054;">
This request and result are similar as the previously request when we added a &lt;container> resource. But when we look at the output in the <a href="start-notificationServer.ipynb">Notification Server</a> notebook, we see that the notification contains the same resource as this request. This way any receiver of a notification can react to a change of a resource without polling a CSE.<br>
This behaviour can be changed when configuring the &lt;subscription> resource.    </div>
</div>
</p>

Any further creation of a &lt;contentInstance> resource will cause a new notification to be sent.


### Requests & Response Attributes

The request and response introduce new attributes:

| Attribute                 | Short Name | Description                                                                                                                                     |
|:--------------------------|:-----------|:------------------------------------------------------------------------------------------------------------------------------------------------|
| notificationEvent | nev | The notification request contains a notification event. |
| representation | rep | Representation for the notification as it was configured with the &lt;subscription>'s <i>notificationContentType</i> attribute. |


&nbsp;